<a href="https://colab.research.google.com/github/EoghanPower/pr/blob/main/Copy_of_Create_a_Tweets_DataFramefinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#Creating a Data Frame of Tweets on S&P 500 Companies

In [2]:
# import libraries
import tweepy, json 
from textblob import TextBlob 
from wordcloud import wordcloud
import pandas as pd
import numpy as np 
import re
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import webbrowser
import time 
import requests
import requests_oauthlib
import os

In [3]:
#authentication store 0auth credentials in variables
consumer_key = 'eHrpqTZVq5bfGdjC8juN7fSCs'
consumer_secret = 'itfojeERaMFXtwQFI1emQNT3vwCHs414OJdOXlpN2RaBAw3MIz'
AccessToken = '264845011-WhC9gKJlC3TayNsXg4ViBIuwaK0D5Sof3lxPTZeN'
AccessTokenSecret = 'dE5difJJmYP5UYllqeQwTsovcoXyvco9Ww2QMxiR365D3'

In [4]:
#Pass 0auth details to tweepy's 0auth handler 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(AccessToken, AccessTokenSecret)

In [5]:
##SetAPIobject
api = tweepy.API(auth, wait_on_rate_limit= True)

In [6]:
#StreamListener
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")

    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + '\n' )
        self.num_tweets += 1
        if self.num_tweets < 10001:
            return True
        else:
            return False
        self.file.close()

    def on_error(self, status):
        print(status)

In [7]:
#initialize stream listener
l = MyStreamListener()
#Create stream object with authentication
stream = tweepy.Stream(auth, l)
#Filter Twitter Streams to capture data by the keywords:
stream.filter(languages = ['en'],track=['Apple', 'Microsoft', 'Google', 'Amazon',  'Facebook', 'Tesla', 'JPMorgan',  'Johnson and Johnson', 'Home Depot','Walmart'])

In [8]:
#import package
import json

#String of path to file: tweets_data_path
tweets_data_path = 'tweets.txt'

#initialize empty list to store tweets: tweets_data
tweets_data = []

In [9]:
#open connection to file
tweets_file = open(tweets_data_path, 'r')

In [10]:
# Read in tweets and store in list: tweets_data
for line in tweets_file:
  tweet = json.loads(line)
  tweets_data.append(tweet)

In [11]:
#close connection to file
tweets_file.close()


In [12]:
print(tweets_data[0].keys())

dict_keys(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])


In [13]:
# Build DataFrame of tweet texts and languages
df = pd.DataFrame(tweets_data, columns= ['text'])

In [14]:
print(df.head(10))

                                                text
0  @elonmusk I successfully bought a Tesla using ...
1  $MSFT Microsoft (MSFT) initiated with an Outpe...
2  RT @WORLDMUSICAWARD: Korean Superstar Group #A...
3  RT @elonmusk: Lots of Tesla cars to deliver be...
4  RT @JoshuaPotash: Today Amazon workers in Chic...
5  RT @alexnpress: one of these workers told HR a...
6  RT @pete3291: Justin Welby defending Johnson a...
7                             @Lpyou_ google flights
8  @iiiiiiiiiix2 @elonmusk Because of inflation, ...
9  RT @BLACKPINKSTATS5: .@BLACKPINK's "THE ALBUM"...


In [15]:
#clean the tweets
def cleanTwt(twt): 
  twt = re.sub('#apple','Apple', twt)
  twt = re.sub('#microsoft','Microsoft', twt) 
  twt = re.sub('#google','Google', twt) 
  twt = re.sub('#amazon','Amazon', twt) 
  twt = re.sub('#facebook','Facebook', twt) 
  twt = re.sub('#Tesla ','Tesla', twt)
  twt = re.sub('#JPMorgan','JPMorgan', twt) 
  twt = re.sub('#JohnsonandJohnson','Johnson and Johnson', twt) 
  twt = re.sub('#HomeDepot','HomeDepot', twt) 
  twt = re.sub('#Walmart','Walmart', twt) #removes the # 
  twt =re.sub('#[A-Za-z0-9]+', '',twt) #removes any strings with a # 
  twt = re.sub('\\n', '', twt) #removes the '\n' string
  twt = re.sub('https?:\/\/\S+', '', twt) #removes any hyperlinks
  twt = re.sub('RT', '', twt) #cleans retweets
  twt = re.sub('@[A-Za-z0-9]+', ',', twt)#Removes tagged accounts
  return twt

In [16]:
#clean them using the above function

In [17]:
df['Cleaned_Tweets'] = df['text'].apply(cleanTwt)

In [18]:
#show the set 
df.head(10)

,text,Cleaned_Tweets
0,@elonmusk I successfully bought a Tesla using ...,", I successfully bought a Tesla using btc😛"
1,$MSFT Microsoft (MSFT) initiated with an Outpe...,$MSFT Microsoft (MSFT) initiated with an Outpe...
2,RT @WORLDMUSICAWARD: Korean Superstar Group #A...,",: Korean Superstar Group scores this week's..."
3,RT @elonmusk: Lots of Tesla cars to deliver be...,",: Lots of Tesla cars to deliver before year ..."
4,RT @JoshuaPotash: Today Amazon workers in Chic...,",: Today Amazon workers in Chicago walked off..."
5,RT @alexnpress: one of these workers told HR a...,",: one of these workers told HR and managemen..."
6,RT @pete3291: Justin Welby defending Johnson a...,",: Justin Welby defending Johnson and the che..."
7,@Lpyou_ google flights,",_ google flights"
8,"@iiiiiiiiiix2 @elonmusk Because of inflation, ...",", , Because of inflation, shortages, and the f..."
9,"RT @BLACKPINKSTATS5: .@BLACKPINK's ""THE ALBUM""...",",: .,'s ""THE ALBUM"" is now the longest charti..."


In [19]:
#create a function to get subjectivity
def getSubjectivity(twt):
  return TextBlob(twt).sentiment.subjectivity
#create a function to get polarity
def getPolarity(twt):
  return TextBlob(twt).sentiment.polarity


In [20]:
#create two new columns called subjectivity and polarity
df['Subjectivity'] = df['Cleaned_Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Cleaned_Tweets'].apply(getPolarity)

In [21]:
#show data
df.head(20)

,text,Cleaned_Tweets,Subjectivity,Polarity
0,@elonmusk I successfully bought a Tesla using ...,", I successfully bought a Tesla using btc😛",0.950000,0.7500
1,$MSFT Microsoft (MSFT) initiated with an Outpe...,$MSFT Microsoft (MSFT) initiated with an Outpe...,0.000000,0.0000
2,RT @WORLDMUSICAWARD: Korean Superstar Group #A...,",: Korean Superstar Group scores this week's...",0.000000,0.0000
3,RT @elonmusk: Lots of Tesla cars to deliver be...,",: Lots of Tesla cars to deliver before year ...",0.150000,0.2000
4,RT @JoshuaPotash: Today Amazon workers in Chic...,",: Today Amazon workers in Chicago walked off...",0.500000,0.5000
5,RT @alexnpress: one of these workers told HR a...,",: one of these workers told HR and managemen...",0.400000,0.2000
6,RT @pete3291: Justin Welby defending Johnson a...,",: Justin Welby defending Johnson and the che...",0.000000,0.0000
7,@Lpyou_ google flights,",_ google flights",0.000000,0.0000
8,"@iiiiiiiiiix2 @elonmusk Because of inflation, ...",", , Because of inflation, shortages, and the f...",0.500000,0.2500
9,"RT @BLACKPINKSTATS5: .@BLACKPINK's ""THE ALBUM""...",",: .,'s ""THE ALBUM"" is now the longest charti...",0.000000,0.0000


In [22]:
#Create a function to get the sentiment text
def getSentiment(score):
  if score <0:
    return 'Negative'
  elif score == 0:
      return 'Neutral'
  else:
      return 'Positive'

In [23]:
#Create column to store the text sentiment
df['Sentiment'] = df['Polarity'].apply(getSentiment)
#Show the data
df.head(7)

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,@elonmusk I successfully bought a Tesla using ...,", I successfully bought a Tesla using btc😛",0.95,0.75,Positive
1,$MSFT Microsoft (MSFT) initiated with an Outpe...,$MSFT Microsoft (MSFT) initiated with an Outpe...,0.00,0.00,Neutral
2,RT @WORLDMUSICAWARD: Korean Superstar Group #A...,",: Korean Superstar Group scores this week's...",0.00,0.00,Neutral
3,RT @elonmusk: Lots of Tesla cars to deliver be...,",: Lots of Tesla cars to deliver before year ...",0.15,0.20,Positive
4,RT @JoshuaPotash: Today Amazon workers in Chic...,",: Today Amazon workers in Chicago walked off...",0.50,0.50,Positive
5,RT @alexnpress: one of these workers told HR a...,",: one of these workers told HR and managemen...",0.40,0.20,Positive
6,RT @pete3291: Justin Welby defending Johnson a...,",: Justin Welby defending Johnson and the che...",0.00,0.00,Neutral


In [24]:
df.drop(columns = 'text')

,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,", I successfully bought a Tesla using btc😛",0.950000,0.750000,Positive
1,$MSFT Microsoft (MSFT) initiated with an Outpe...,0.000000,0.000000,Neutral
2,",: Korean Superstar Group scores this week's...",0.000000,0.000000,Neutral
3,",: Lots of Tesla cars to deliver before year ...",0.150000,0.200000,Positive
4,",: Today Amazon workers in Chicago walked off...",0.500000,0.500000,Positive
...,...,...,...,...
9995,",: *heads to Amazon*",0.000000,0.000000,Neutral
9996,",_studio: 💥$100,000 WAVES Giveaway💥To Enter:✅...",0.000000,0.000000,Neutral
9997,", Wow!!! This is a game changer. Keep retweeti...",0.700000,-0.214844,Negative
9998,This is a funny headline. The number of revie...,0.533333,0.125000,Positive


In [25]:
#drop columns to make it easier to read
df.head()

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,@elonmusk I successfully bought a Tesla using ...,", I successfully bought a Tesla using btc😛",0.95,0.75,Positive
1,$MSFT Microsoft (MSFT) initiated with an Outpe...,$MSFT Microsoft (MSFT) initiated with an Outpe...,0.00,0.00,Neutral
2,RT @WORLDMUSICAWARD: Korean Superstar Group #A...,",: Korean Superstar Group scores this week's...",0.00,0.00,Neutral
3,RT @elonmusk: Lots of Tesla cars to deliver be...,",: Lots of Tesla cars to deliver before year ...",0.15,0.20,Positive
4,RT @JoshuaPotash: Today Amazon workers in Chic...,",: Today Amazon workers in Chicago walked off...",0.50,0.50,Positive


In [26]:
df.tail()

,text,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
9995,RT @LateefSaka: *heads to Amazon*,",: *heads to Amazon*",0.000000,0.000000,Neutral
9996,"RT @melos_studio: 💥$100,000 WAVES Giveaway💥\n\...",",_studio: 💥$100,000 WAVES Giveaway💥To Enter:✅...",0.000000,0.000000,Neutral
9997,@cherthedev Wow!!! This is a game changer. Kee...,", Wow!!! This is a game changer. Keep retweeti...",0.700000,-0.214844,Negative
9998,This is a funny headline. The number of revie...,This is a funny headline. The number of revie...,0.533333,0.125000,Positive
9999,RT @sha_sports: Congratulations to @ValkyriesB...,",_sports: Congratulations to , on winning the...",0.750000,0.625000,Positive


In [27]:
#export the dataframe to sheets
#export to google sheets part one authentication
!pip install gspread
from google.colab import auth as au
au.authenticate_user()

In [28]:
!pip install df2gspread

     |████████████████████████████████| 56 kB 2.5 MB/s 
  Created wheel for df2gspread: filename=df2gspread-1.0.4-py3-none-any.whl size=11952 sha256=31d4da0f285d54066b24917e1b5414c9355e43435dc51abf8f514e61571f13c1
  Stored in directory: /root/.cache/pip/wheels/4d/78/20/cf9a2810f93885ad71bd351c8b604455ba160ac4b0e9b7d526
Successfully built df2gspread
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 4.4.0 requires google-api-python-client>=1.7.8, but you have google-api-python-client 1.6.7 which is incompatible.
earthengine-api 0.1.290 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.6.7 whi

In [29]:
import gspread
from oauth2client.client import GoogleCredentials

In [30]:
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [31]:
from gspread.models import Worksheet
#export to google sheets part two export
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [32]:
from df2gspread import df2gspread as d2g

In [33]:
spreadsheet_key = '1IjzY-rTedrKHQNz9PahAeCkqZ3zF-Z6h-GiMw8zysNY'
wks_name = 'Master'
d2g.upload(df, spreadsheet_key, wks_name, credentials= GoogleCredentials.get_application_default(), row_names=True)

<Worksheet 'Master' id:1124026790>